## Import necessary libraries

In [22]:
%reset -f

import pickle
import pandas as pd
import numpy
import time
from sklearn.neural_network import MLPClassifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer

## Save environment parameters, so those won't be saved to report file

In [23]:
environment_variables = dir()

## Define model related constants

In [24]:
import numpy

# Input parameters
INPUT_TRAINING_FILE = "../data/original/training_dataset.csv"
INPUT_EVALUATION_FILE = "../data/original/evaluation_dataset.csv"

# Output parameters
METHOD_NAME = "multi_layer_perceptron"
TIMESTAMP = time.strftime("%d_%m_%Y-%H_%M_%S")
OUTPUT_MODEL = f"../data/models/{METHOD_NAME}_model_{TIMESTAMP}.pkl"
OUTPUT_RESULTS = f"../data/results/{METHOD_NAME}_model_{TIMESTAMP}.txt"

# Hyper parameter alternatives
HYPER_PARAMETER_K_FEATURES = list(range(30, 90, 10))
HYPER_PARAMETER_SCORE_FUNC = [chi2, f_classif]
HYPER_PARAMETER_HIDDEN_LAYER_SIZES = [(100,)]
#HYPER_PARAMETER_HIDDEN_LAYER_SIZES = [(100,), (100, 100), (100, 100, 100),
#    (200,), (200, 200), (200, 200, 200), (100, 500)]
HYPER_PARAMETER_ALPHA = list(10.0 ** -numpy.arange(1, 7))

# Hyper parameter optimization parameters
HYPER_PARAMETER_OPTIMIZATION_BETA = 2
HYPER_PARAMETER_OPTIMIZATION_CV = 5

# Other constants
LABELS = ["Operational", "Bankrupt"]
RANDOM_SEED = 42
VERBOSITY = 3

## Load the datasets
These dataset should contain "Bankrupt?" label and features that should be considered during learning.

In [25]:
training_dataset = pd.read_csv(INPUT_TRAINING_FILE, engine="python", delimiter=",")
training_features = training_dataset.loc[:, training_dataset.columns != "Bankrupt?"]
training_targets = training_dataset["Bankrupt?"]

evaluation_dataset = pd.read_csv(INPUT_EVALUATION_FILE, engine="python", delimiter=",")
evaluation_features = evaluation_dataset.loc[:, evaluation_dataset.columns != "Bankrupt?"]
evaluation_targets = evaluation_dataset["Bankrupt?"]

## Minor preprocessing
Remove constant Net Income Flag and  Liability-Assets Flag features.

In [26]:
training_features = training_features.drop(columns=[" Net Income Flag"])
evaluation_features = evaluation_features.drop(columns=[" Net Income Flag"])
training_features = training_features.drop(columns=[" Liability-Assets Flag"])
evaluation_features = evaluation_features.drop(columns=[" Liability-Assets Flag"])

## Define the training pipeline

In [27]:
# Create internal pipeline
classifier = MLPClassifier(solver="lbfgs")
standard_scaler =  StandardScaler()
min_max_scaler =  MinMaxScaler()
feature_selection = SelectKBest()
# pca = PCA(random_state=RANDOM_SEED)
smote = SMOTE(sampling_strategy="minority", random_state=RANDOM_SEED)
pipeline = Pipeline(steps=[("standard_scaler", standard_scaler),("min_max_scaler", min_max_scaler),
    ("selection", feature_selection), ("smote", smote), ("classification", classifier)])

# Specify the tunable hyper parameters
parameters = {
    "selection__k": HYPER_PARAMETER_K_FEATURES,
    "selection__score_func": HYPER_PARAMETER_SCORE_FUNC,
    "classification__hidden_layer_sizes": HYPER_PARAMETER_HIDDEN_LAYER_SIZES,
    "classification__alpha": HYPER_PARAMETER_ALPHA
}

# Define KFold parameters
cv = StratifiedKFold(n_splits=HYPER_PARAMETER_OPTIMIZATION_CV, shuffle=True, random_state=RANDOM_SEED)

# Define custom fbeta scorer function that put emphasis on recall
def custom_scorer(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=HYPER_PARAMETER_OPTIMIZATION_BETA)

estimator = GridSearchCV(pipeline, parameters, verbose=VERBOSITY,
    scoring=make_scorer(custom_scorer), cv=cv, n_jobs=-1)

## Do training

In [28]:
estimator.fit(training_features, training_targets)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/home/roni/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 3/5] END classification__alpha=0.1, classification__hidden_layer_sizes=(100,), selection__k=30, selection__score_func=<function chi2 at 0x7f0948c9d040>;, score=0.437 total time=   7.6s


/home/roni/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 1/5] END classification__alpha=0.1, classification__hidden_layer_sizes=(100,), selection__k=30, selection__score_func=<function chi2 at 0x7f0614e0f040>;, score=0.512 total time=   8.4s


...

[CV 3/5] END classification__alpha=1e-06, classification__hidden_layer_sizes=(100,), selection__k=80, selection__score_func=<function f_classif at 0x7f0614ebdee0>;, score=0.444 total time=   8.0s


/home/roni/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 5/5] END classification__alpha=1e-06, classification__hidden_layer_sizes=(100,), selection__k=80, selection__score_func=<function f_classif at 0x7fe37c905ee0>;, score=0.556 total time=   7.2s


/home/roni/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('standard_scaler', StandardScaler()),
                                       ('min_max_scaler', MinMaxScaler()),
                                       ('selection', SelectKBest()),
                                       ('smote',
                                        SMOTE(random_state=42,
                                              sampling_strategy='minority')),
                                       ('classification',
                                        MLPClassifier(solver='lbfgs'))]),
             n_jobs=-1,
             param_grid={'classification__alpha': [0.1, 0.01, 0.001, 0.0001,
                                                   1e-05, 1e-06],
                         'classification__hidden_layer_sizes': [(100,)],
                         'selection__k': [30, 40, 50, 60, 70, 80],
                         'selection__score_func': [<function chi2 at 0x7f

## Calculate metric values

In [29]:
def confusion_matrix_to_string(cm):
    return f"True operational={cm[0][0]}, True bankrupt={cm[1][1]}, False operational={cm[0][1]}, False bankrupt={cm[1][0]}"

# Use all training data to calculate confusion matrix for training data
training_estimates = estimator.predict(training_features)
training_accuracy = balanced_accuracy_score(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix_to_string(training_confusion_matrix)
training_classification_report = classification_report(training_targets, training_estimates, output_dict=True, target_names=LABELS)
training_f_beta_score = fbeta_score(training_targets, training_estimates, beta=HYPER_PARAMETER_OPTIMIZATION_BETA)

# Use model to estimate manually labeled evaluation Tweets
evaluation_estimates = estimator.predict(evaluation_features)
evaluation_accuracy = balanced_accuracy_score(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix_to_string(evaluation_confusion_matrix)
evaluation_classification_report = classification_report(evaluation_targets, evaluation_estimates, output_dict=True, target_names=LABELS)
evaluation_f_beta_score = fbeta_score(evaluation_targets, evaluation_estimates, beta=HYPER_PARAMETER_OPTIMIZATION_BETA)

# Store best parameters
best_parameters = estimator.best_params_
print(best_parameters)

{'classification__alpha': 1e-06, 'classification__hidden_layer_sizes': (100,), 'selection__k': 40, 'selection__score_func': <function f_classif at 0x7fa25811a280>}


## Save trained model for future reference

In [30]:
with open(OUTPUT_MODEL, "wb") as handle:
    pickle.dump(estimator, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Save metadata and calculated statistics
This is done by saving every textual and numerical variable to file.

In [31]:
current_variables = dir()

results_string = ""

for variable in current_variables:
    # Skip environment variables and their container variable
    # Ignore also underscore variables
    if variable in environment_variables or variable == "environment_variables" or variable.startswith("_"):
        continue

    # Get variables value
    variable_value = globals()[variable]

    # If variable is numerical or string, append it to results
    if type(variable_value) is str or type(variable_value) is int or \
        type(variable_value) is float or type(variable_value) is list or \
        type(variable_value) is numpy.float64 or type(variable_value) is dict:
        results_string += f"{variable}: {variable_value}\n"

# Print results to screen
print(results_string)

# Save results to file
with open(OUTPUT_RESULTS, "w") as file:
    file.write(results_string)

HYPER_PARAMETER_ALPHA: [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06]
HYPER_PARAMETER_HIDDEN_LAYER_SIZES: [(100,)]
HYPER_PARAMETER_K_FEATURES: [30, 40, 50, 60, 70, 80]
HYPER_PARAMETER_OPTIMIZATION_BETA: 2
HYPER_PARAMETER_OPTIMIZATION_CV: 5
HYPER_PARAMETER_SCORE_FUNC: [<function chi2 at 0x7fa25811a3a0>, <function f_classif at 0x7fa25811a280>]
INPUT_EVALUATION_FILE: ../data/original/evaluation_dataset.csv
INPUT_TRAINING_FILE: ../data/original/training_dataset.csv
LABELS: ['Operational', 'Bankrupt']
METHOD_NAME: multi_layer_perceptron
OUTPUT_MODEL: ../data/models/multi_layer_perceptron_model_11_05_2022-16_42_48.pkl
OUTPUT_RESULTS: ../data/results/multi_layer_perceptron_model_11_05_2022-16_42_48.txt
RANDOM_SEED: 42
TIMESTAMP: 11_05_2022-16_42_48
VERBOSITY: 3
best_parameters: {'classification__alpha': 1e-06, 'classification__hidden_layer_sizes': (100,), 'selection__k': 40, 'selection__score_func': <function f_classif at 0x7fa25811a280>}
evaluation_accuracy: 0.8037878787878787
evaluation_classific